In [8]:
from aco import ACO
import numpy as np
from scipy.spatial import distance_matrix
import logging
from sklearn.preprocessing import StandardScaler

N_ANTS = 30
N_ITERATIONS = [1, 5, 10, 20, 40, 60, 80, 100]



def heuristics_reevo(edge_attr):
    num_edges = edge_attr.shape[0]
    num_attributes = edge_attr.shape[1]

    heuristic_values = np.zeros_like(edge_attr)

    # Apply feature engineering on edge attributes
    transformed_attr = np.log1p(np.abs(edge_attr))  # Taking logarithm of absolute value of attributes
    
    # Normalize edge attributes
    scaler = StandardScaler()
    edge_attr_norm = scaler.fit_transform(transformed_attr)

    # Calculate correlation coefficients
    correlation_matrix = np.corrcoef(edge_attr_norm.T)

    # Calculate heuristic value for each edge attribute
    for i in range(num_edges):
        for j in range(num_attributes):
            if edge_attr_norm[i][j] != 0:
                heuristic_values[i][j] = np.exp(-8 * edge_attr_norm[i][j] * correlation_matrix[j][j])

    return heuristic_values


def solve(node_pos):
    dist_mat = distance_matrix(node_pos, node_pos)
    dist_mat[np.diag_indices_from(dist_mat)] = 1 # set diagonal to a large number
    heu = heuristics_reevo(dist_mat.copy()) + 1e-9
    heu[heu < 1e-9] = 1e-9
    aco = ACO(dist_mat, heu, n_ants=N_ANTS)
    
    results = []
    for i in range(len(N_ITERATIONS)):
        if i == 0:
            obj = aco.run(N_ITERATIONS[i])
        else:
            obj = aco.run(N_ITERATIONS[i] - N_ITERATIONS[i-1])
        # print("Iteration: {}, Objective: {}".format(N_ITERATIONS[i], obj))
        results.append(obj.item())
    return results

In [10]:
print("[*] Running ...")

for problem_size in [20, 50, 100]:
    dataset_path = f"./dataset/test{problem_size}_dataset.npy"
    node_positions = np.load(dataset_path)
    logging.info(f"[*] Evaluating {dataset_path}")
    n_instances = node_positions.shape[0]
    objs = []
    for i, node_pos in enumerate(node_positions):
        obj = solve(node_pos)
        objs.append(obj)
    # Average objective value for all instances
    mean_obj = np.mean(objs, axis=0)
    for i, obj in enumerate(mean_obj):
        print(f"[*] Average for {problem_size}, {N_ITERATIONS[i]} iterations: {obj}")
    print()

[*] Running ...
[*] Average for 20, 1 iterations: 3.9003731003737356
[*] Average for 20, 5 iterations: 3.8462281953062325
[*] Average for 20, 10 iterations: 3.835403331472863
[*] Average for 20, 20 iterations: 3.829248570223732
[*] Average for 20, 40 iterations: 3.82684029390981
[*] Average for 20, 60 iterations: 3.8268176874268875
[*] Average for 20, 80 iterations: 3.826708686712287
[*] Average for 20, 100 iterations: 3.826708686712287

[*] Average for 50, 1 iterations: 6.275422895093018
[*] Average for 50, 5 iterations: 5.990559736071067
[*] Average for 50, 10 iterations: 5.9140437101383165
[*] Average for 50, 20 iterations: 5.868818218520994
[*] Average for 50, 40 iterations: 5.842789093344989
[*] Average for 50, 60 iterations: 5.826968493451817
[*] Average for 50, 80 iterations: 5.814727992597635
[*] Average for 50, 100 iterations: 5.808828626754723

[*] Average for 100, 1 iterations: 9.419730651353163
[*] Average for 100, 5 iterations: 8.787484845631024
[*] Average for 100, 10 ite